## Imports

In [36]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler

## Functions

In [37]:
def load_data(data_path):
    df = pd.read_csv(data_path)
    return df


def preprocess_data(df):
    df["color_change"] = df["color"].diff().ne(0).astype(int)
    df["color_change"].fillna(0, inplace=True)

    return df


def scale_data(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled


def timeseries_cv_score(X, y, n_splits):
    tscv = TimeSeriesSplit(n_splits=n_splits)

    f1_scores = []
    auc_scores = []  # list to store ROC AUC scores for each split
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Define LSTM model
        model = Sequential()
        model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
        model.add(Dense(1, activation="sigmoid"))  # because of binary classification

        model.compile(
            loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
        )

        # Train the model
        model.fit(X_train, y_train, epochs=10, verbose=0)

        # Make predictions on the test set
        y_pred = model.predict(X_test).ravel()

        # Calculate F1 score of the model on the test set
        f1 = f1_score(y_test, (y_pred > 0.5).astype("int32"))
        f1_scores.append(f1)

        # Calculate ROC AUC score of the model on the test set
        auc = roc_auc_score(y_test, y_pred)
        auc_scores.append(auc)

    return np.mean(f1_scores), np.mean(auc_scores)

## Globals

In [38]:
# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Load the data
data_path = "../../../data/kc/btc/raw/kc_btc_15min.csv"

## Preprocessing

In [46]:
df = load_data(data_path)

# Preprocess the data
df = preprocess_data(df)

# Prepare TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

## FEATURE PROCESSING ##

df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# print(df.index.duplicated().any())

df = df.loc[~df.index.duplicated(keep="first")]

print(df.ta.categories)

df.ta.strategy("all")

# Check your results and exclude as necessary.
df.ta.strategy(fast=10, slow=50, verbose=True)

# Sanity check. Make sure all the columns are there
print(df.columns)

# Forward Fill
df.ffill(inplace=True)

# Backward Fill
df.bfill(inplace=True)

# Re-scale the data to include the new feature
df_scaled = scale_data(df)

X = df.drop("color_change", axis=1)
y = df["color_change"]

df.tail()
# print(df.isna().sum())

['candles', 'cycles', 'momentum', 'overlap', 'performance', 'statistics', 'trend', 'volatility', 'volume']


131it [00:02, 52.87it/s]


[+] Strategy: All
[i] Indicator arguments: {'fast': 10, 'slow': 50, 'append': True}
[i] Excluded[12]: above, above_value, below, below_value, cross, cross_value, long_run, short_run, td_seq, tsignals, vp, xsignals
[i] Multiprocessing 131 indicators with 3 chunks and 12/12 cpus.


131it [00:02, 49.57it/s]

[i] Total indicators: 131
[i] Columns added: 30
[i] Last Run: Tuesday June 20, 2023, NYSE: 20:16:45, Local: 0:16:45 Pacific Daylight Time, Day 171/365 (47.00%)
Index(['open', 'close', 'high', 'low', 'volume', 'color', 'color_change',
       'ABER_ZG_5_15', 'ABER_SG_5_15', 'ABER_XG_5_15',
       ...
       'PVOs_10_50_9', 'SMI_10_50_5', 'SMIs_10_50_5', 'SMIo_10_50_5',
       'STC_10_10_50_0.5', 'STCmacd_10_10_50_0.5', 'STCstoch_10_10_50_0.5',
       'TSI_10_50_13', 'TSIs_10_50_13', 'UO_10_14_50'],
      dtype='object', length=315)


,open,close,high,low,volume,color,color_change,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,ACCBM_20,ACCBU_20,AD,ADOSC_3_10,ADX_14,DMP_14,DMN_14,ALMA_10_6.0_0.85,AMATe_LR_8_21_2,AMATe_SR_8_21_2,AO_5_34,OBV,OBV_min_2,OBV_max_2,OBVe_4,OBVe_12,AOBV_LR_2,AOBV_SR_2,APO_12_26,AROOND_14,AROONU_14,AROONOSC_14,ATRr_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,BIAS_SMA_26,BOP,AR_26,BR_26,CCI_14_0.015,CDL_2CROWS,CDL_3BLACKCROWS,CDL_3INSIDE,CDL_3LINESTRIKE,CDL_3OUTSIDE,CDL_3STARSINSOUTH,CDL_3WHITESOLDIERS,CDL_ABANDONEDBABY,CDL_ADVANCEBLOCK,CDL_BELTHOLD,CDL_BREAKAWAY,CDL_CLOSINGMARUBOZU,CDL_CONCEALBABYSWALL,CDL_COUNTERATTACK,CDL_DARKCLOUDCOVER,CDL_DOJI_10_0.1,CDL_DOJISTAR,CDL_DRAGONFLYDOJI,CDL_ENGULFING,CDL_EVENINGDOJISTAR,CDL_EVENINGSTAR,CDL_GAPSIDESIDEWHITE,CDL_GRAVESTONEDOJI,CDL_HAMMER,CDL_HANGINGMAN,CDL_HARAMI,CDL_HARAMICROSS,CDL_HIGHWAVE,CDL_HIKKAKE,CDL_HIKKAKEMOD,CDL_HOMINGPIGEON,CDL_IDENTICAL3CROWS,CDL_INNECK,CDL_INSIDE,CDL_INVERTEDHAMMER,CDL_KICKING,CDL_KICKINGBYLENGTH,CDL_LADDERBOTTOM,CDL_LONGLEGGEDDOJI,CDL_LONGLINE,CDL_MARUBOZU,CDL_MATCHINGLOW,CDL_MATHOLD,CDL_MORNINGDOJISTAR,CDL_MORNINGSTAR,CDL_ONNECK,CDL_PIERCING,CDL_RICKSHAWMAN,CDL_RISEFALL3METHODS,CDL_SEPARATINGLINES,CDL_SHOOTINGSTAR,CDL_SHORTLINE,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS,open_Z_30_1,high_Z_30_1,low_Z_30_1,close_Z_30_1,CFO_9,CG_10,CHOP_14_1_100,CKSPl_10_3_20,CKSPs_10_3_20,CMF_20,CMO_14,COPC_11_14_10,CTI_12,LDECAY_5,DEC_1,DEMA_10,DCL_20_20,DCM_20_20,DCU_20_20,DPO_20,EBSW_40_10,EFI_13,EMA_10,ENTP_10,EOM_14_100000000,ER_10,BULLP_13,BEARP_13,FISHERT_9_1,FISHERTs_9_1,FWMA_10,HA_open,HA_high,HA_low,HA_close,HILO_13_21,HILOl_13_21,HILOs_13_21,HL2,HLC3,HMA_10,HWM,HWU,HWL,HWMA_0.2_0.1_0.1,ISA_9,ISB_26,ITS_9,IKS_26,ICS_26,INC_1,INERTIA_20_14,JMA_7_0,KAMA_10_2_30,KCLe_20_2,KCBe_20_2,KCUe_20_2,K_9_3,D_9_3,J_9_3,KST_10_15_20_30_10_10_10_15,KSTs_9,KURT_30,KVO_34_55_13,KVOs_34_55_13,LR_14,LOGRET_1,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MAD_30,MASSI_9_25,MCGD_10,MEDIAN_30,MFI_14,MIDPOINT_2,MIDPRICE_2,MOM_10,NATR_14,NVI_1,OHLC4,PDIST,PCTRET_1,PGO_14,PPO_12_26_9,PPOh_12_26_9,PPOs_12_26_9,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2,PSL_12,PVI_1,PVO_12_26_9,PVOh_12_26_9,PVOs_12_26_9,PVOL,PVR,PVT,PWMA_10,QQE_14_5_4.236,QQE_14_5_4.236_RSIMA,QQEl_14_5_4.236,QQEs_14_5_4.236,QS_10,QTL_30_0.5,RMA_10,ROC_10,RSI_14,RSX_14,RVGI_14_4,RVGIs_14_4,RVI_14,SINWMA_14,SKEW_30,SLOPE_1,SMA_10,SMI_5_20_5,SMIs_5_20_5,SMIo_5_20_5,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO,SQZPRO_20_2.0_20_2_1.5_1,SQZPRO_ON_WIDE,SQZPRO_ON_NORMAL,SQZPRO_ON_NARROW,SQZPRO_OFF,SQZPRO_NO,SSF_10_2,STC_10_12_26_0.5,STCmacd_10_12_26_0.5,STCstoch_10_12_26_0.5,STDEV_30,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0,SWMA_10,T3_10_0.7,TEMA_10,THERMO_20_2_0.5,THERMOma_20_2_0.5,THERMOl_20_2_0.5,THERMOs_20_2_0.5,TOS_STDEVALL_LR,TOS_STDEVALL_L_1,TOS_STDEVALL_U_1,TOS_STDEVALL_L_2,TOS_STDEVALL_U_2,TOS_STDEVALL_L_3,TOS_STDEVALL_U_3,TRIMA_10,TRIX_30_9,TRIXs_30_9,TRUERANGE_1,TSI_13_25_13,TSIs_13_25_13,TTM_TRND_6,UI_14,UO_7_14_28,VAR_30,VHF_28,VIDYA_14,VTXP_14,VTXM_14,VWAP_D,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30,ADOSC_10_50,AMATe_LR_10_50_2,AMATe_SR_10_50_2,AO_10_50,OBVe_10,OBVe_50,APO_10_50,COPC_10_50_10,KAMA_10_10_50,KVO_10_50_13,KVOs_10_50_13,MACD_10_50_9,MACDh_10_50_9,MACDs_10_50_9,MASSI_10_50,PPO_10_50_9,PPOh_10_50_9,PPOs_10_50_9,PVO_10_50_9,PVOh_10_50_9,PVOs_10_50_9,SMI_10_50_5,SMIs_10_50_5,SMIo_10_50_5,STC_10_10_50_0.5,STCmacd_10_10_50_0.5,STCstoch_10_10_50_0.5,TSI_10_50_13,TSIs_10_50_13,UO_10_14_50
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

## Univariate Feature Selection Process

In [40]:
# Re-scale the data to include the new feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# feature selection
selector = SelectKBest(score_func=f_classif, k="all")
selector.fit(X_scaled, y)

# Get columns to keep and create new dataframe with those only
cols = selector.get_support(indices=True)
features_df_new = X.iloc[:, cols]

# Store the scores of each feature in a dictionary
feature_scores = {
    feature_name: score for feature_name, score in zip(X.columns, selector.scores_)
}

# Sort the dictionary by value in descending order and print the scores
for feature_name, score in sorted(
    feature_scores.items(), key=lambda item: item[1], reverse=True
):
    print(f"{feature_name}: {score}")

# Now we can apply Logistic Regression and Random Forests on the new features_df_new
# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, features_df_new, y, cv=tscv, scoring="f1")

print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, features_df_new, y, cv=tscv, scoring="f1")

print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_f1_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV F1 score: {mean_f1_score}")

print("\n", features_df_new.columns)

c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 45  49  51  54  56  57  76  79  80  81  86  89 100 103 104] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\jnorm\Projects\websocket_trading\.venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


THERMOs_20_2_0.5: 273.60362108048645
volume: 91.47185887138416
BBB_5_2.0: 79.13500197982871
CDL_IDENTICAL3CROWS: 22.908645526841614
CDL_HANGINGMAN: 19.791770412852728
DMN_14: 11.907639542415298
AROONU_14: 9.647634376719118
DMP_14: 8.054813920069284
AROOND_14: 3.6653826739854787
AMATe_LR_8_21_2: 1.2859566508917912
AMATe_SR_8_21_2: 1.065560252719997
ABER_ATR_5_15: 0.7788502422916445
ADOSC_3_10: 0.6883592208277559
OBV_min_2: 0.5591854195833879
OBVe_12: 0.4798570820528162
OBVe_4: 0.4771745840486053
color: 0.4452904087414569
OBV: 0.41619803294575647
OBV_max_2: 0.34522088140368123
low: 0.30409701012902224
ALMA_10_6.0_0.85: 0.2976418393705159
AO_5_34: 0.28576718207771223
ABER_XG_5_15: 0.275972186704456
ABER_ZG_5_15: 0.2552379590935276
ACCBL_20: 0.24436544363719015
ABER_SG_5_15: 0.23539790386047119
open: 0.22470195021284903
close: 0.2124001771743569
ACCBM_20: 0.2110670855398578
AD: 0.20718604906113675
ACCBU_20: 0.1777763069827223
APO_12_26: 0.17109364723535517
ADX_14: 0.16572563017629532
high:

## Base Model

In [41]:
# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")

print(f"Logistic Regression CV ROC AUC score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")

print(f"Random Forest CV ROC AUC score: {np.mean(cv_scores)}")

print("\n", X.columns)

Logistic Regression CV ROC AUC score: 0.5517433276575191
Random Forest CV ROC AUC score: 0.8991939050899015

 Index(['open', 'close', 'high', 'low', 'volume', 'color', 'ABER_ZG_5_15',
       'ABER_SG_5_15', 'ABER_XG_5_15', 'ABER_ATR_5_15',
       ...
       'VIDYA_14', 'VTXP_14', 'VTXM_14', 'VWAP_D', 'VWMA_10', 'WCP',
       'WILLR_14', 'WMA_10', 'ZL_EMA_10', 'ZS_30'],
      dtype='object', length=284)


## LSTM

In [42]:
# Reshape input to be 3D [samples, timesteps, features]
X_array = X.values
X_reshaped = X_array.reshape((X_array.shape[0], 1, X_array.shape[1]))

# Call the function
mean_auc_score = timeseries_cv_score(X_reshaped, y.values, n_splits=5)
print(f"\nLSTM CV ROC AUC score: {mean_auc_score}")

42/42 [==============================] - 0s 647us/step

LSTM CV ROC AUC score: (0.6959629098786181, 0.4926542232731929)
